# Create entry points to spark


In [1]:
!pip3 install pyspark

    100% |████████████████████████████████| 215.7MB 115kB/s 
    100% |████████████████████████████████| 204kB 29.1MB/s 
  Stored in directory: /root/.cache/pip/wheels/47/9b/57/7984bf19763749a13eece44c3174adb6ae4bc95b920375ff50
Successfully built pyspark


In [0]:
try:
  sc.stop()
except:
  pass
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
sc=SparkContext()
spark=SparkSession(sparkContext=sc)

# RDD object
The class **pyspark.SparkContext** creates a client which connects to a Spark cluster. This client can be used to create an RDD object. There are two methods from this class for directly creating RDD objects:



*   parallelize()
*   textFile()


# parallelize()
parallelize() distribute a **local python collection** to form an RDD. Common built-in python collections include dist, list, tuple or set.



In [4]:
rdd=sc.parallelize([1,2,3])
rdd.collect()

[1, 2, 3]

In [5]:
# from a list of tuple
list_t = [('cat', 'dog', 'fish'), ('orange', 'apple')]
rdd = sc.parallelize(list_t)
rdd.collect()

[('cat', 'dog', 'fish'), ('orange', 'apple')]

In [6]:
# from a set
s = {'cat', 'dog', 'fish', 'cat', 'dog', 'dog'}
rdd = sc.parallelize(s)
rdd.collect()

['fish', 'cat', 'dog']

In [7]:

# from a dict
d = {
    'a': 100,
    'b': 200,
    'c': 300
}
rdd = sc.parallelize(d)
rdd.collect()

['a', 'b', 'c']

# textFile()

The textFile() function reads a text file and returns it as an RDD of strings. Usually, you will need to apply some map functions to transform each elements of the RDD to some data structure/type that is suitable for data analysis.

**When using textFile(), each line of the text file becomes an element in the resulting RDD.**


In [31]:
rddh=sc.textFile("sample_data/california_housing_train.csv")
rddh.take(5)

['"longitude","latitude","housing_median_age","total_rooms","total_bedrooms","population","households","median_income","median_house_value"',
 '-114.310000,34.190000,15.000000,5612.000000,1283.000000,1015.000000,472.000000,1.493600,66900.000000',
 '-114.470000,34.400000,19.000000,7650.000000,1901.000000,1129.000000,463.000000,1.820000,80100.000000',
 '-114.560000,33.690000,17.000000,720.000000,174.000000,333.000000,117.000000,1.650900,85700.000000',
 '-114.570000,33.640000,14.000000,1501.000000,337.000000,515.000000,226.000000,3.191700,73400.000000']


# DataFrame object
# Creat DataFrame by reading a file

In [10]:
houseing_data=spark.read.csv(path="sample_data/california_housing_train.csv",sep=",",encoding="UTF-8",comment=None,header=True,inferSchema=True)
houseing_data.show(5,truncate=False)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|-114.31  |34.19   |15.0              |5612.0     |1283.0        |1015.0    |472.0     |1.4936       |66900.0           |
|-114.47  |34.4    |19.0              |7650.0     |1901.0        |1129.0    |463.0     |1.82         |80100.0           |
|-114.56  |33.69   |17.0              |720.0      |174.0         |333.0     |117.0     |1.6509       |85700.0           |
|-114.57  |33.64   |14.0              |1501.0     |337.0         |515.0     |226.0     |3.1917       |73400.0           |
|-114.57  |33.57   |20.0              |1454.0     |326.0         |624.0     |262.0     |1.925        |65500.0           |
+---------+--------+----

# Create DataFrame with createDataFrame function

# From an RDD

Elements in RDD has to be an Row object

In [11]:
from pyspark.sql import Row
rdd=sc.parallelize([Row(x=[1,2,3],y=["a","b","c"]),Row(x=[4,5,6],y=["g","h","i"])])
rdd.collect()

[Row(x=[1, 2, 3], y=['a', 'b', 'c']), Row(x=[4, 5, 6], y=['g', 'h', 'i'])]

In [12]:
df=spark.createDataFrame(rdd)
df.show()

+---------+---------+
|        x|        y|
+---------+---------+
|[1, 2, 3]|[a, b, c]|
|[4, 5, 6]|[g, h, i]|
+---------+---------+



# From pandas DataFrame


In [13]:
import pandas as pd
house=pd.read_csv("sample_data/california_housing_train.csv")
house.head()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
0,-114.31,34.19,15.0,5612.0,1283.0,1015.0,472.0,1.4936,66900.0
1,-114.47,34.40,19.0,7650.0,1901.0,1129.0,463.0,1.8200,80100.0
2,-114.56,33.69,17.0,720.0,174.0,333.0,117.0,1.6509,85700.0
3,-114.57,33.64,14.0,1501.0,337.0,515.0,226.0,3.1917,73400.0
4,-114.57,33.57,20.0,1454.0,326.0,624.0,262.0,1.9250,65500.0


In [14]:
house_spark=spark.createDataFrame(house)
house_spark.show()

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+
|  -114.31|   34.19|              15.0|     5612.0|        1283.0|    1015.0|     472.0|       1.4936|           66900.0|
|  -114.47|    34.4|              19.0|     7650.0|        1901.0|    1129.0|     463.0|         1.82|           80100.0|
|  -114.56|   33.69|              17.0|      720.0|         174.0|     333.0|     117.0|       1.6509|           85700.0|
|  -114.57|   33.64|              14.0|     1501.0|         337.0|     515.0|     226.0|       3.1917|           73400.0|
|  -114.57|   33.57|              20.0|     1454.0|         326.0|     624.0|     262.0|        1.925|           65500.0|
|  -114.58|   33.63|    

# Column instance

Column instances can be created in two ways:



*   directly select a column out of a DataFrame: **df.colName**
*   create from a column expression: **df.colName + 1**




**Remember how to create column instances, because this is usually the starting point if we want to operate DataFrame columns.**

The column classes come with some methods that can operate on a column instance. However, almost all functions from ***the pyspark.sql.functions*** module take one or more column instances as argument(s). These functions are important for data manipulation tools.



# DataFrame column methods


# Methods that take column names as arguments:


*   corr(col1, col2): two column names
*   cov(col1, col2): two column names.

*   describe(*cols): `cols` refers to only column names (strings).*

*   crosstab(col1, col2): two column names.




# Methods that take column names or column expressions or both as arguments:



*  cube(*cols): column names (string) or column expressions or both.
*   drop(*cols): a list of column names OR a single column expression.





*   groupBy(*cols): column name (string) or column expression or both.
*  rollup(*cols): column name (string) or column expression or both.



*  toDF(*cols): a list of column names (string).
*   filter(condition): *condition refers to a column expression that returns types.BooleanType of values.







# DataFrame to RDD
A DataFrame can be easily converted to an RDD by calling the pyspark.sql.DataFrame.rdd() function. Each element in the returned RDD is an **pyspark.sql.Row object**. An Row is a list of key-value pairs.


In [15]:
house_spark.rdd.take(2)

[Row(longitude=-114.31, latitude=34.19, housing_median_age=15.0, total_rooms=5612.0, total_bedrooms=1283.0, population=1015.0, households=472.0, median_income=1.4936, median_house_value=66900.0),
 Row(longitude=-114.47, latitude=34.4, housing_median_age=19.0, total_rooms=7650.0, total_bedrooms=1901.0, population=1129.0, households=463.0, median_income=1.82, median_house_value=80100.0)]


With an RDD object, we can apply a set of mapping functions, such as **map, mapValues, flatMap, flatMapValues** and a lot of other methods that come from RDD.

In [19]:
house_spark_map=house_spark.rdd.map(lambda x: (x["longitude"],x["latitude"]))
house_spark_map.take(5)

[(-114.31, 34.19),
 (-114.47, 34.4),
 (-114.56, 33.69),
 (-114.57, 33.64),
 (-114.57, 33.57)]

In [24]:
house_spark_mapvalues=house_spark_map.mapValues(lambda x:[x,x*10])
house_spark_mapvalues.take(5)

[(-114.31, [34.19, 341.9]),
 (-114.47, [34.4, 344.0]),
 (-114.56, [33.69, 336.9]),
 (-114.57, [33.64, 336.4]),
 (-114.57, [33.57, 335.7])]

# RDD to DataFrame
To convert an RDD to a DataFrame, we can use the SparkSession.createDataFrame() function. Every element in the RDD has be to an Row object.



In [25]:
raw_rdd=house_spark.rdd
raw_rdd.take(5)

[Row(longitude=-114.31, latitude=34.19, housing_median_age=15.0, total_rooms=5612.0, total_bedrooms=1283.0, population=1015.0, households=472.0, median_income=1.4936, median_house_value=66900.0),
 Row(longitude=-114.47, latitude=34.4, housing_median_age=19.0, total_rooms=7650.0, total_bedrooms=1901.0, population=1129.0, households=463.0, median_income=1.82, median_house_value=80100.0),
 Row(longitude=-114.56, latitude=33.69, housing_median_age=17.0, total_rooms=720.0, total_bedrooms=174.0, population=333.0, households=117.0, median_income=1.6509, median_house_value=85700.0),
 Row(longitude=-114.57, latitude=33.64, housing_median_age=14.0, total_rooms=1501.0, total_bedrooms=337.0, population=515.0, households=226.0, median_income=3.1917, median_house_value=73400.0),
 Row(longitude=-114.57, latitude=33.57, housing_median_age=20.0, total_rooms=1454.0, total_bedrooms=326.0, population=624.0, households=262.0, median_income=1.925, median_house_value=65500.0)]

In [34]:
rddh.take(5)

['"longitude","latitude","housing_median_age","total_rooms","total_bedrooms","population","households","median_income","median_house_value"',
 '-114.310000,34.190000,15.000000,5612.000000,1283.000000,1015.000000,472.000000,1.493600,66900.000000',
 '-114.470000,34.400000,19.000000,7650.000000,1901.000000,1129.000000,463.000000,1.820000,80100.000000',
 '-114.560000,33.690000,17.000000,720.000000,174.000000,333.000000,117.000000,1.650900,85700.000000',
 '-114.570000,33.640000,14.000000,1501.000000,337.000000,515.000000,226.000000,3.191700,73400.000000']

In [38]:
header=rddh.map(lambda x:x.split(",")).collect()[0]
header

['"longitude"',
 '"latitude"',
 '"housing_median_age"',
 '"total_rooms"',
 '"total_bedrooms"',
 '"population"',
 '"households"',
 '"median_income"',
 '"median_house_value"']

In [47]:
rdd = rddh.map(lambda x: x.split(',')).filter(lambda line: line != header)
rdd.take(2)

[['-114.310000',
  '34.190000',
  '15.000000',
  '5612.000000',
  '1283.000000',
  '1015.000000',
  '472.000000',
  '1.493600',
  '66900.000000'],
 ['-114.470000',
  '34.400000',
  '19.000000',
  '7650.000000',
  '1901.000000',
  '1129.000000',
  '463.000000',
  '1.820000',
  '80100.000000']]


# Convert RDD elements to RDD Row objects
First we define a function which takes a list of column names and a list of values and create a Row of key-value pairs. Since keys in an Row object are variable names, we can’t simply pass a dictionary to the Row() function. We can think of a dictionary as an argument list and use the ** to unpack the argument list.

In [48]:
from pyspark.sql import Row
my_dict = dict(zip(['a', 'b', 'c'], range(1, 4)))
Row(**my_dict)

Row(a=1, b=2, c=3)

In [0]:
def list_to_row(keys, values):
    row_dict = dict(zip(keys, values))
    return Row(**row_dict)

In [50]:
rdd_rows = rdd.map(lambda x: list_to_row(header, x))
rdd_rows.take(3)

[Row("households"='472.000000', "housing_median_age"='15.000000', "latitude"='34.190000', "longitude"='-114.310000', "median_house_value"='66900.000000', "median_income"='1.493600', "population"='1015.000000', "total_bedrooms"='1283.000000', "total_rooms"='5612.000000'),
 Row("households"='463.000000', "housing_median_age"='19.000000', "latitude"='34.400000', "longitude"='-114.470000', "median_house_value"='80100.000000', "median_income"='1.820000', "population"='1129.000000', "total_bedrooms"='1901.000000', "total_rooms"='7650.000000'),
 Row("households"='117.000000', "housing_median_age"='17.000000', "latitude"='33.690000', "longitude"='-114.560000', "median_house_value"='85700.000000', "median_income"='1.650900', "population"='333.000000', "total_bedrooms"='174.000000', "total_rooms"='720.000000')]

In [51]:
df1 = spark.createDataFrame(rdd_rows)
df1.show(5)

+------------+--------------------+----------+-----------+--------------------+---------------+------------+----------------+-------------+
|"households"|"housing_median_age"|"latitude"|"longitude"|"median_house_value"|"median_income"|"population"|"total_bedrooms"|"total_rooms"|
+------------+--------------------+----------+-----------+--------------------+---------------+------------+----------------+-------------+
|  472.000000|           15.000000| 34.190000|-114.310000|        66900.000000|       1.493600| 1015.000000|     1283.000000|  5612.000000|
|  463.000000|           19.000000| 34.400000|-114.470000|        80100.000000|       1.820000| 1129.000000|     1901.000000|  7650.000000|
|  117.000000|           17.000000| 33.690000|-114.560000|        85700.000000|       1.650900|  333.000000|      174.000000|   720.000000|
|  226.000000|           14.000000| 33.640000|-114.570000|        73400.000000|       3.191700|  515.000000|      337.000000|  1501.000000|
|  262.000000|      

# merge-and-split-columns

# Merge and split columns
Sometimes we need to merge multiple columns in a Dataframe into one column, or split a column into multiple columns. We can easily achieve this by converting a DataFrame to RDD, applying map functions to manipulate elements, and then converting the RDD back to a DataFrame.

In [53]:
colnames = df1.columns
colnames[0] = 'households_number'
household=df1.rdd.toDF(colnames)
household.show(5)

+-----------------+--------------------+----------+-----------+--------------------+---------------+------------+----------------+-------------+
|households_number|"housing_median_age"|"latitude"|"longitude"|"median_house_value"|"median_income"|"population"|"total_bedrooms"|"total_rooms"|
+-----------------+--------------------+----------+-----------+--------------------+---------------+------------+----------------+-------------+
|       472.000000|           15.000000| 34.190000|-114.310000|        66900.000000|       1.493600| 1015.000000|     1283.000000|  5612.000000|
|       463.000000|           19.000000| 34.400000|-114.470000|        80100.000000|       1.820000| 1129.000000|     1901.000000|  7650.000000|
|       117.000000|           17.000000| 33.690000|-114.560000|        85700.000000|       1.650900|  333.000000|      174.000000|   720.000000|
|       226.000000|           14.000000| 33.640000|-114.570000|        73400.000000|       3.191700|  515.000000|      337.000000|

In [54]:
from pyspark.sql import Row
value_rdd = df1.rdd.map(lambda x: Row(model=x[0], values=x[1:]))
value_rdd.take(5)
value_df = spark.createDataFrame(value_rdd)
value_df.show(5, truncate=False)

+----------+--------------------------------------------------------------------------------------------------+
|model     |values                                                                                            |
+----------+--------------------------------------------------------------------------------------------------+
|472.000000|[15.000000, 34.190000, -114.310000, 66900.000000, 1.493600, 1015.000000, 1283.000000, 5612.000000]|
|463.000000|[19.000000, 34.400000, -114.470000, 80100.000000, 1.820000, 1129.000000, 1901.000000, 7650.000000]|
|117.000000|[17.000000, 33.690000, -114.560000, 85700.000000, 1.650900, 333.000000, 174.000000, 720.000000]   |
|226.000000|[14.000000, 33.640000, -114.570000, 73400.000000, 3.191700, 515.000000, 337.000000, 1501.000000]  |
|262.000000|[20.000000, 33.570000, -114.570000, 65500.000000, 1.925000, 624.000000, 326.000000, 1454.000000]  |
+----------+--------------------------------------------------------------------------------------------

# Split one column
We use the above DataFrame as our example data. Again, we need to convert the DataFrame to an RDD to achieve our goal.

Let's split the values column into two columns: x1 and x2. The first 4 values will be in column x1 and the remaining values will be in column x2.

In [56]:
value_rdd_2 = value_df.rdd.map(lambda x: Row(model=x[0], x1=x[1][:5], x2=x[1][5:]))
# convert RDD back to DataFrame
value_df_2 = spark.createDataFrame(value_rdd_2)
value_df_2.show(5, truncate=False)

+----------+-----------------------------------------------------------+---------------------------------------+
|model     |x1                                                         |x2                                     |
+----------+-----------------------------------------------------------+---------------------------------------+
|472.000000|[15.000000, 34.190000, -114.310000, 66900.000000, 1.493600]|[1015.000000, 1283.000000, 5612.000000]|
|463.000000|[19.000000, 34.400000, -114.470000, 80100.000000, 1.820000]|[1129.000000, 1901.000000, 7650.000000]|
|117.000000|[17.000000, 33.690000, -114.560000, 85700.000000, 1.650900]|[333.000000, 174.000000, 720.000000]   |
|226.000000|[14.000000, 33.640000, -114.570000, 73400.000000, 3.191700]|[515.000000, 337.000000, 1501.000000]  |
|262.000000|[20.000000, 33.570000, -114.570000, 65500.000000, 1.925000]|[624.000000, 326.000000, 1454.000000]  |
+----------+-----------------------------------------------------------+------------------------